In [24]:
import pandas as pd 
import numpy as np 
import neo4j


import seaborn as sns
import warnings
from neo4j import GraphDatabase
warnings.filterwarnings('ignore')
##from pandas_profiling import ProfileReport

In [27]:
start_node = neo4j.data.Node("User", id=user_id)

AttributeError: module 'neo4j.data' has no attribute 'Node'

In [14]:
url = "neo4j+s://9d0eb28d.databases.neo4j.io"
username = "neo4j"
password = ""

driver = GraphDatabase.driver(url, auth=(username, password))
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n)")
    count = result.single()[0]
    print(f"La conexión con Aura Neo4j se ha establecido correctamente. Hay {count} nodos en la base de datos.")

La conexión con Aura Neo4j se ha establecido correctamente. Hay 83 nodos en la base de datos.


In [21]:
query = """
MATCH (u:User {id: $user_id})-[:IN_VISTAS]->(m:Movie)
RETURN m.id AS movie_id
"""

user_id = 666
with driver.session() as session:
    result = session.run(query, user_id=user_id)
    vistas = [record["movie_id"] for record in result]
    
query = f"MATCH (m:Movie) WHERE NOT m.id IN {vistas} RETURN m.id AS movie_id"
with driver.session() as session:
    result = session.run(query)
    no_vistas = [record["movie_id"] for record in result]
    
result = session.run(
            """
            CALL algo.pageRank.stream(
                'MATCH (m:Movie)-[:IN_GENRE]->(g:Genre) RETURN id(m) as id',
                'MATCH (m1:Movie)-[:SIMILAR_TO]-(m2:Movie) RETURN id(m1) as source, id(m2) as target',
                {
                    sourceNodes: $start_node,
                    relationshipWeightProperty: 'weight',
                    writeProperty: 'pagerank'
                }
            )
            """,
            start_node="MATCH (u:User {id: 1})",
        )

